plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

## Pseudocode:
    . Import
    . Define functions
    . Input symptom, mode, perdur
    . Load Parameters
    . For Loop through seizures

## Import

In [1]:
import csv
import os
import pathlib
import numpy as np
from scipy.io import loadmat
from scipy.signal import butter,filtfilt

#
import pandas as pd
import datetime
# from plot_change_module import *

## Define Functions

In [ ]:
def ll_sort(

In [2]:
def ll_transform(llw,fs,d,bl_start,bl_stop):
    L = int(np.round(llw * fs) - 1) # number of samples to calculate line length
    LL = np.empty(np.shape(d))
    LL[:] = np.NaN    
    
    sample_bl_start = max([fs*bl_start,1])[0]
    sample_bl_end = (fs*bl_stop)[0]
    
    # Line-length transform
    for row_1 in range(0,3):
    # for row_1 in range(0,np.shape(d)[0]):
        for col_1 in range(0,np.shape(d)[1]-L):
            LL[row_1,col_1] = np.sum(np.abs(np.diff(d.iloc[row_1,col_1:col_1+L])))
            
        LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
        LL_nanstd = np.nanstd(LL[row_1,sample_bl_start:sample_bl_end])

        # Normalize LL (to baseline period) as z-scores, "zLL"
        LL[row_1,:] = (LL[row_1,:] - LL_nanmean)/LL_nanstd
    return LL

In [3]:
def open_xl(xl_name): 
    # df = pd.read_excel(xl_name)
    # Ndimensions = df.shape #row, col
    # params_list = df.values.tolist()
    
    df = pd.read_excel(xl_name, index_col=0, engine='openpyxl')

    return df


In [4]:
def filt_perdur(sx_input,mx_input,perdur_input,d,sem_df,fs):
    d_t = d.transpose()
    
    butter_array = np.array([1,(round(fs[0]/2)-1)])
    b,a = butter(2,butter_array/(fs[0]/2),btype='bandpass',output='ba')#,analog = True)
    
    try:
        mx_index = sem_df.index[sem_df[sx_input]==int(mx_input)][0] # first onset of symptom in ppEEG time 
        onset_mx = round((mx_index/5)*fs[0]) #adjusted for ppEEG time
        
        ppEEG_perdur = int(perdur_input) * fs[0]

        # onset_series = d.iloc[onset_mx]
        before_series = d_t.iloc[onset_mx - ppEEG_perdur]
        after_series = d_t.iloc[onset_mx + ppEEG_perdur]
        
        # onset_array = onset_series.to_numpy()
        before_array = before_series.to_numpy()
        after_array = after_series.to_numpy()

        # onset_array = filtfilt(b,a,~np.isnan(onset_array),axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)) # d(nns,:) = filtfilt(b,a,d(nns,:)')'; 
        before_array = filtfilt(b,a,~np.isnan(before_array),axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose() # d(nns,:) = filtfilt(b,a,d(nns,:)')'; 
        after_array = filtfilt(b,a,~np.isnan(after_array),axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose() # d(nns,:) = filtfilt(b,a,d(nns,:)')'; 
        
        return before_array, after_array
    
    except IndexError:
          print('no index or ppEEG')


In [5]:
def load_elecs_anat(pt_path):
                
    #load anatomy and electrode matrix
    os.chdir(pt_path + 'Imaging/elecs')
    e_mat = loadmat('clinical_elecs_all.mat')
    anatomy = e_mat['anatomy']
    em = e_mat['elecmatrix']
    em_row = np.shape(anatomy)[0]
    
    return em_row, em, anatomy


In [6]:
def get_params(df_params,pt):
    params_bl_start = df_params.loc[pt]['BLstart']
    params_bl_stop = df_params.loc[pt]['BLstop']
    params_llw = df_params.loc[pt]['llw']

    return params_bl_start, params_bl_stop, params_llw

In [7]:
def get_ptsz(avg_change_path): #Load list of patient and seizure names
    os.chdir(avg_change_path)
    
    df_params = open_xl('OPSCEAparams.xlsx') # Ndimensions and params_list

    with open('sz_name_list.csv','r') as sz_names:
        sz_names_list = list(csv.reader(sz_names))
        sz_rows = np.size(sz_names_list,0) #find row num 
        pt_data = []
        sz_data= []
        for r in range(0,sz_rows):  #collect symptom data for specfic seizure in list
            pt_data.append(sz_names_list[r][0]) 
            sz_data.append(sz_names_list[r][1]) 
    return pt_data, sz_data, df_params

## Input symptom, mode, and duration

In [8]:
sx_input = 'lhx'
mx_input = '2'
perdur_input = '10'

In [9]:
neuroanat_list = ['frontalpole', #FRONTAL LOBE
    'parstriangularis',
    'parsopercularis',
    'parsorbitalis',
    'rostralmiddlefrontal',
    'caudalmiddlefrontal',
    'lateralorbitofrontal',
    'superiorfrontal',
    'medialorbitofrontal',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inferiorparietal',   
    'superiorparietal',
    'supramarginal',
    'temporalpole', # TEMPORAL LOBE
    'middletemporal',
    'superiortemporal',
    'inferiortemporal',
    'parahippocampal',               
    'Right-Hippocampus',
    'Left-Hippocampus',
    'Right-Amygdala',
    'Left-Amygdala',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual',
    'Right-Inf-Lat-Vent', # OTHER
    'Right-Cerebral-White-Matter',
    'Left-Cerebral-White-Matter',
    'Right-choroid-plexus',
    'Right-Putamen',
    'Right-VentralDC'];

## For loop through patients in list matching seizure

In [10]:
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'

pt_data,sz_data,df_params = get_ptsz(avg_change_path) #Ehsan

# for neuro_struc in neuroanat_list:

#For loop through patients
for i in range(len(pt_data)):  #Ehsan
    pt_i = pt_data[i]          #Ehsan
    sz_name = sz_data[i]       #Ehsan

    pt_path = avg_change_path + pt_i + '/'
    os.chdir(pt_path)
    pt_dir = os.listdir(pt_path)

    for sz_name in sz_data: 

        if sz_name in pt_dir: #patients in list matching seizure
 
            print(sz_name)
    
            blstart, blstop, llw = get_params(df_params,pt_i) # 2 = params_llw
                                    
            sz_path = pt_path + sz_data[i]
            
            os.chdir(sz_path)

            #load data
            sem_df = pd.read_csv(sz_name + '_mat.csv',usecols = [sx_input]) #load semiology matrix .csv
            
            sz_mat = loadmat(sz_name +'.mat') #load frame speed and electrode matrix    
            fs = sz_mat['fs'].flatten()
            ppEEG = sz_mat['ppEEG']
            
            d = pd.DataFrame(ppEEG)
            
            #remove EKG lines at end of anatomy + ppEEG?

            try:
                filt_before, filt_after = filt_perdur(sx_input,mx_input,perdur_input,d,sem_df,fs) #creates 2 arrays of before and after seizure onset and filters
            except:
                print('no filt_perdur')

            try:
                LL = ll_transform(llw,fs,d,blstart,blstop)                        
            except:
                print('no ll_transform')
            
            # try:
            #     LL_s = ll_sort(LL)
            # except:
            #     print('no ll_sort')



EC96_01
[[0.53024869 0.58089842 0.59201849 ...        nan        nan        nan]
 [0.38574534 0.23376138 0.1884797  ...        nan        nan        nan]
 [2.62646081 2.50410516 2.53804051 ...        nan        nan        nan]
 ...
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]]
EC107_01
[[-1.40104469 -1.41295205 -1.44149059 ...         nan         nan
          nan]
 [-1.08909966 -1.04002926 -1.02424979 ...         nan         nan
          nan]
 [-0.64273316 -0.55033496 -0.45319918 ...         nan         nan
          nan]
 ...
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]]
EC133_03
no index or ppEEG
no filt_perdur


/var/folders/yg/mdppj78s1zg98yqntmdjd3xw0000gn/T/ipykernel_98114/2687851925.py:15: RuntimeWarning: Mean of empty slice
  LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
EC229_02
[[-0.80948794 -0.81571281 -0.81531306 ...         nan         nan
          nan]
 [-1.20470364 -1.23572045 -1.2299156  ...         nan         nan
          nan]
 [-1.32389018 -1.35819444 -1.37459255 ...         nan         nan
          nan]
 ...
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]]
